In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import sys

sys.path.append("incl/")

import ELPH_dyn
import ELPH_utils


#global hyperparameters
kmax = 4.
n_kmax = 80


# init = ELPH_dyn.get_init_cond_gauss(kmax = kmax, n_kmax = n_kmax, max_pos = 0.15, width = 0.05, density=density)
# plt.plot(init[:n_kmax])
# plt.show()

# nkdyn = ELPH_dyn.get_el_dynamics(init, n_kmax = n_kmax)
# plt.imshow(nkdyn, aspect='auto')
# plt.colorbar()
# plt.show()

In [ ]:
n_runs = 100

paras = np.zeros((n_runs,3))

paras[:,0] = np.linspace(0.1,0.7,n_runs) #max_pos
paras[:,1] = 0.05 #width
paras[:,2] = 0.1 #density

runs = ELPH_utils.get_runs_gaussian_init(kmax, n_kmax, paras, tmax=2000, n_tmax=400)


run  1  from  100
run  2  from  100
run  3  from  100
run  4  from  100
run  5  from  100
run  6  from  100
run  7  from  100
run  8  from  100
run  9  from  100
run  10  from  100
run  11  from  100
run  12  from  100
run  13  from  100
run  14  from  100
run  15  from  100
run  16  from  100
run  17  from  100
run  18  from  100
run  19  from  100
run  20  from  100
run  21  from  100
run  22  from  100
run  23  from  100
run  24  from  100
run  25  from  100
run  26  from  100
run  27  from  100
run  28  from  100


In [ ]:
# plt.imshow(runs[1], aspect='auto')
# plt.colorbar()
# plt.show()

print('runs[0] shape: ', runs[0].shape)

for k in range(n_runs):
    plt.plot(runs[k][:,500])

plt.title('initial conditions')
plt.show()

In [ ]:
from ELPH_VAR import SVDVAR
    
VAR = SVDVAR(runs, rdim=10, n_VAR_steps=5)

VAR.train(alpha=10**-2.7, rdim = 20, n_VAR_steps = 20, method='ridge')

VAR.print_status()


In [ ]:
plt.semilogy(VAR.S, 'o')
plt.title('singular values off all runs')
plt.show()
 
for l in range(VAR.rdim):
  plt.plot(VAR.U[:,l],  label='Uhat_'+str(l))
plt.title('first ' + str(VAR.rdim) + ' modes')
plt.show()

In [ ]:

run = runs[80]
testpred = VAR.predict_single_run(run)

print(VAR.get_error(run, norm='fro') )

plt.imshow(run, aspect='auto', interpolation='None')
plt.colorbar()
plt.title('truth')
plt.show()

plt.imshow(testpred, aspect='auto', interpolation='None')
plt.colorbar()
plt.title('prediction')
plt.show()

plt.imshow(run-testpred, aspect='auto', interpolation='None')
plt.colorbar()
plt.title('diffrence: prediction - test')
plt.xlim(0,20)
plt.show()

In [ ]:
train_kwargs = {'alpha':10**-2.7, 'rdim':20, 'n_VAR_steps':20}
score_kwargs = {'norm':'max', 'errSVD':False}

mean_score, scores = ELPH_utils.get_KFold_CV_scores(VAR, runs, folds=5, seed=817, score_kwargs=score_kwargs, train_kwargs=train_kwargs)

print('mean score: ', mean_score)
print('scores: ', scores)


In [ ]:
def get_BO_score(aExp, rdim, n_VAR_steps):
    alpha = 10.0**aExp
    rdim = int(rdim)
    n_VAR_steps = int(n_VAR_steps)
    
    train_kwargs = {'alpha':alpha, 'rdim':rdim, 'n_VAR_steps':n_VAR_steps}
    score_kwargs = {'norm':'fro', 'errSVD':False}
    
    m,s = ELPH_utils.get_KFold_CV_scores(VAR, runs, folds=5, seed=817, score_kwargs=score_kwargs, train_kwargs=train_kwargs)
    return -1.*m - 1.*np.std(s)


# print(KFCV_score(-6,3,5))

from bayes_opt import BayesianOptimization

# Bounded region of parameter space
pbounds = {'aExp': [-6, -1], 'rdim': [1,20], 'n_VAR_steps': [1,20]}

optimizer = BayesianOptimization(
    f=get_BO_score,
    pbounds=pbounds,
    random_state=816,
)

# optimizer.maximize(init_points=10, n_iter=100)

In [ ]:

from ELPH_NVAR import SVDNVAR


ntestrun = 32
train_runs = runs.copy()
test_run = train_runs.pop(ntestrun)

a = 1e-2
r = 5
n = 3


NVAR = SVDNVAR(train_runs)

NVAR.train(alpha=a, rdim=r, n_VAR_steps=n, NVAR_p=2)

NVAR.print_status()

plt.imshow(NVAR.w, aspect='auto', interpolation='none')
plt.colorbar()
plt.title('NVAR weights')
plt.show()

plt.semilogy(np.sort(np.ravel(np.abs(NVAR.w))))
plt.title('sorted NVAR weights')
plt.show()


print('NVAR err: ', NVAR.get_error(test_run, norm='fro') )
trNVAR = NVAR.predict_single_run(test_run)

plt.imshow(trNVAR, aspect='auto')
plt.colorbar()
plt.xlim(0,100)
plt.show()


VAR.load_runs(train_runs)
VAR.train(alpha=a, rdim=r, n_VAR_steps=n)

VAR.print_status()
print('VAR err: ', VAR.get_error(test_run, norm='fro') )
trVAR = VAR.predict_single_run(test_run)






In [ ]:
train_runs = runs[::1]
test_run = train_runs.pop(45)

NVAR.load_runs(train_runs)

NVAR.train(rdim=20, n_VAR_steps=2, NVAR_p=2, method='stlsq', alpha=1e-3, threshold=0, intercept=True, standardize=False)

NVAR.print_status()

masked_weights = np.ma.masked_values(NVAR.w,0.0)

plt.imshow(masked_weights, aspect='auto', interpolation='none')
plt.colorbar()
plt.title('NVAR weights')
plt.show()

plt.semilogy(np.sort(np.ravel(np.abs(NVAR.w))))
plt.title('sorted NVAR weights')
plt.show()


print('NVAR err: ', NVAR.get_error(test_run, norm='fro') )
preNVAR = NVAR.predict_single_run(test_run)

plt.imshow(preNVAR, aspect='auto')
plt.colorbar()
plt.title('prediction')
# plt.xlim(0,100)
plt.show()

plt.imshow(preNVAR-test_run, aspect='auto')
plt.colorbar()
plt.title('difference')
# plt.xlim(0,100)
plt.show()

In [ ]:
def get_BayO_score(aExp, thrExp):
    alpha = 10.0**aExp
    threshold = 10**thrExp
    
    train_kwargs = {'alpha':alpha, 'threshold':threshold, 'rdim':20, 'n_VAR_steps':2, 'NVAR_p':2, 'method':'stlsq', 'intercept':True, 'standardize':False}
    score_kwargs = {'norm':'fro', 'errSVD':False}
    
    m,s = ELPH_utils.get_KFold_CV_scores(NVAR, runs, folds=5, seed=817, score_kwargs=score_kwargs, train_kwargs=train_kwargs)
    return -1.*m - 1.*np.std(s)

# print(get_BayO_score(-3,-5))

from bayes_opt import BayesianOptimization
# Bounded region of parameter space
pbounds = {'aExp': [-4, -1], 'thrExp': [-6,-3]}
optimizer = BayesianOptimization(f=get_BayO_score, pbounds=pbounds, random_state=817)

# optimizer.maximize(init_points=10, n_iter=100)

In [ ]:
train_kwargs = {'alpha':10**-3.5, 'threshold':10**-3.0, 'rdim':20, 'n_VAR_steps':2, 'NVAR_p':2, 'method':'stlsq', 'intercept':True, 'standardize':False}
score_kwargs = {'norm':'fro', 'errSVD':False}

m,s = ELPH_utils.get_KFold_CV_scores(NVAR, runs, folds=5, seed=817, score_kwargs=score_kwargs, train_kwargs=train_kwargs)
print(m)
print(s)

In [ ]:
def get_BayO_score(aExp):
    alpha = 10.0**aExp
    
    train_kwargs = {'alpha':alpha, 'rdim':20, 'n_VAR_steps':2, 'NVAR_p':2, 'method':'ridge', 'intercept':True, 'standardize':False}
    score_kwargs = {'norm':'fro', 'errSVD':False}
    
    m,s = ELPH_utils.get_KFold_CV_scores(NVAR, runs, folds=5, seed=817, score_kwargs=score_kwargs, train_kwargs=train_kwargs)
    return -1.*m - 1.*np.std(s)

# print(get_BayO_score(-3))

from bayes_opt import BayesianOptimization
# Bounded region of parameter space
pbounds = {'aExp': [-4, -1]}
optimizer = BayesianOptimization(f=get_BayO_score, pbounds=pbounds, random_state=817)

# optimizer.maximize(init_points=5, n_iter=60)

In [ ]:
train_runs = runs[::1]
test_run = train_runs.pop(45)

NVAR.load_runs(train_runs)
NVAR.train(rdim=20, n_VAR_steps=2, NVAR_p=2, method='ridge', alpha=10.**-3.4, intercept=True, standardize=False)

NVAR.print_status()


print('NVAR err: ', NVAR.get_error(test_run, norm='fro') )
print('NVAR err compared to reconstruced run: ', NVAR.get_error(test_run, norm='fro', errSVD=True) )

preNVAR = NVAR.predict_single_run(test_run)
recon_runs = NVAR.Uhat @ NVAR.Uhat.T @ test_run


xmax = 100

plt.imshow(preNVAR, aspect='auto', interpolation='none')
plt.colorbar()
plt.title('prediction')
plt.xlim(0,xmax)
plt.show()

plt.imshow(preNVAR-test_run, aspect='auto', interpolation='none')
plt.colorbar()
plt.title('difference: pred - test')
plt.xlim(0,xmax)
plt.show()

plt.imshow(preNVAR-recon_runs, aspect='auto', interpolation='none')
plt.colorbar()
plt.title('difference: pred - reconstructed test')
plt.xlim(0,xmax)
plt.show()

plt.imshow(preNVAR-recon_runs, aspect='auto', interpolation='none')
plt.colorbar()
plt.title('difference: test - reconstructed test')
plt.xlim(0,xmax)
plt.show()

In [ ]:
train_kwargs = {'alpha':10**-1.0, 'rdim':20, 'n_VAR_steps':2, 'NVAR_p':2, 'method':'ridge', 'intercept':True, 'standardize':False}
score_kwargs = {'norm':'max', 'errSVD':True}

m,s = ELPH_utils.get_KFold_CV_scores(NVAR, runs, folds=5, seed=817, score_kwargs=score_kwargs, train_kwargs=train_kwargs)
print(m)
print(s)

In [ ]:
from SVDAPPRX import SVDAPPRX

SVDapr = SVDAPPRX(runs)
SVDapr.train()

m,s = ELPH_utils.get_KFold_CV_scores(SVDapr, runs, folds=5, seed=817, train_kwargs={'rdim':20}, score_kwargs={ 'norm':'max'})
print(m)
print(s)


In [ ]:
SVD_approx_err = np.zeros(n_kmax)
for rdim in range(n_kmax):
    SVD_approx_err[rdim] = ELPH_utils.get_KFold_CV_scores(SVDapr, runs, folds=5, seed=817, train_kwargs={'rdim':rdim+1}, score_kwargs={ 'norm':'max'})[0]

plt.grid()
plt.plot(np.arange(1,n_kmax+1),SVD_approx_err)
plt.xlabel('reconstruction rank')
plt.ylabel('reconstruction error')
plt.show()

plt.grid()
plt.semilogy(np.arange(1,n_kmax+1),SVD_approx_err)
plt.xlabel('reconstruction rank')
plt.ylabel('reconstruction error')
plt.ylim(1e-4,)
plt.show()